In [0]:
import numpy as np
import tensorflow as tf

from typing import List, Tuple, Union

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, Flatten

# Skipgram

Implementation of the Skipgram algorithm for word embeddings

### Preprocessing

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
cd "/content/drive/My Drive/deep learning (Tommaso)"

/content/drive/.shortcut-targets-by-id/1Uhw8IQungyqnlJEo9yI6DY9h6yWgz8bk/deep learning (Tommaso)


In [0]:
with open("alice.txt", "r") as f:
    raw_corpus = f.readlines()

In [0]:
tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n' + "'")
tokenizer.fit_on_texts(raw_corpus)
corpus = sum((tokenizer.texts_to_sequences(raw_corpus)), [])

In [0]:
voc_size = max(corpus) + 1
corpus_size = len(corpus)

In [7]:
corpus_size, voc_size

(27330, 2568)

### Dataset generation

Skipgram is trained on the following task: given a `window_size`, samples randomly a `word_range` between `1` and `window_size` and tries to guess each word `word_range` around the current word.
So we need to construct a dataset of pairs `(word, target)` for each word in the corpus.

In [0]:
def generate_dataset(
    corpus: List[int],
    window_size: int,
    seed: int = 4322
) -> Tuple[np.ndarray, np.ndarray]:
    rand_generator = np.random.default_rng(seed=seed)
    
    x: List[int] = []
    y: List[int] = []    
    for idx, word in enumerate(corpus):
        word_range = rand_generator.integers(1, window_size, endpoint=True)
        
        surrounding_words = corpus[idx - word_range : idx] + corpus[idx + 1 : idx + word_range + 1]
        x += [word] * len(surrounding_words)
        y += surrounding_words
        
    
    np_x = np.array(x)
    np_y = np.array(y)
    
    return np_x, np_y

In [0]:
window_size = 10
x, y = generate_dataset(corpus, window_size)

### Skipgram implementation

Now we implement the skipgram model, should have quite a simple architecture.

In [13]:
embeddding_size = 100
skipgram = Sequential([
    Embedding(voc_size, embeddding_size, input_length=1),
    Flatten(),
    Dense(voc_size, activation="softmax", use_bias=False, kernel_regularizer="l2")
])

skipgram.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

skipgram.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1, 100)            256800    
_________________________________________________________________
flatten (Flatten)            (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 2568)              256800    
Total params: 513,600
Trainable params: 513,600
Non-trainable params: 0
_________________________________________________________________


In [14]:
_ = skipgram.fit(x, y, batch_size=32, epochs=3)

Epoch 1/3
9410/9410 [==============================] - 72s 8ms/step - loss: 6.8351 - accuracy: 0.0587
Epoch 2/3
9410/9410 [==============================] - 69s 7ms/step - loss: 6.4465 - accuracy: 0.0597
Epoch 3/3
9410/9410 [==============================] - 69s 7ms/step - loss: 6.3623 - accuracy: 0.0592


## Evaluation of embeddings

We first get the embedding layer from the model

In [0]:
embedding_layer = skipgram.layers[0]

We then define a function to retrieve the embedding of a word. Can pass custom tokenizer or embedding_layer, but will otherwise use what we defined
above.

In [0]:
def embed(
    word: str,
    tokenizer: Tokenizer = tokenizer,
    embedding_layer: Embedding = embedding_layer
) -> np.ndarray:
    word_index = tokenizer.texts_to_sequences([word])
    output_tensor = embedding_layer(np.array(word_index))
    return tf.reshape(output_tensor, (-1, 1))

We also define a function that retrieves the closes word to a word vector

In [0]:
def cosine_similarity(
    v: Union[np.ndarray, tf.Tensor],
    w: Union[np.ndarray, tf.Tensor]) -> float:
    """v and w should be 1D-vectors, for which to calculate the cosine similarity"""
    
    assert isinstance(v, tf.Tensor) or isinstance(v, np.ndarray), "v must be tf.Tensor or np.ndarray"
    assert isinstance(w, tf.Tensor) or isinstance(w, np.ndarray), "w must be tf.Tensor or np.ndarray"
    
    v = v.numpy() if isinstance(v, tf.Tensor) else v
    w = w.numpy() if isinstance(w, tf.Tensor) else w
    
    if v.ndim > 1:
        v = v.flatten()
    if w.ndim > 1:
        w = w.flatten()

    return v @ w.T / (np.linalg.norm(v) * np.linalg.norm(w))
    
def closest_n_words(
    embedding: tf.Tensor,
    n: int = 1,
    tokenizer: Tokenizer = tokenizer,
    embedding_layer: Embedding = embedding_layer
) -> List[str]:    
    all_words = np.array(list(tokenizer.index_word.values()))
    all_embeddings = np.array([embed(word).numpy() for word in all_words]).squeeze()
    similarities = np.apply_along_axis(
        lambda row: cosine_similarity(row, embedding),
        1, all_embeddings
    )
    sorted_idx = np.argsort(similarities)
    
    return np.flip(all_words[sorted_idx[-n:]]).tolist()

Ideally, we might think that $e_\text{king} - e_\text{man} \approx e_\text{queen} - e_\text{woman}$. So if we try and find the closest word to 
$e_\text{king} - e_\text{queen} + e_\text{woman}$ it should be close to $e_\text{man}$

In [0]:
analogy = embed("king") - embed("queen") + embed("woman")

In [0]:
closest_n_words(analogy, 10)

['old',
 'remarked',
 'your',
 'myself',
 'talk',
 'are',
 'd',
 'believe',
 'we',
 'show']

In [0]:
cosine_similarity(analogy, embed("man"))

0.98229545

In [0]:
all_words = np.array(list(tokenizer.index_word.values()))
all_embeddings = np.array([embed(word).numpy() for word in all_words]).squeeze()

In [0]:
str_embeddings = all_embeddings.astype("str")

In [0]:
with open("skipgrams_embeddings.tsv", "w") as f:
    for line in str_embeddings:
        f.write("\t".join(line) + "\n")

In [0]:
with open("skipgram_meta.tsv", "w") as f:
    for word in all_words:
        f.write(f"{word}\n")